In [25]:
import pandas as pd

In [26]:
# A list containing the AOIs considered in the study --- TO UPDATE IF YOU UPDATE THE AOIs
AOIS = ['Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4',
       'Paragraph5', 'Paragraph6', 'Text_Area', 'SubFigure1', 'SubFigure2',
       'SubFigure3', 'Figure', 'Formula']

In [27]:
# Read gaze data with events and AOIs using pandas library
data = pd.read_csv("data/gazeDataWithAOIsAndEvents.csv")
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [28]:
# Preview data
data.head()

,Row,Timestamp,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,128.9448,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,137.2724,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,500.0,567.9048,16.1935,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,145.6203,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,566.2654,4.7572,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,2.6143,-1369.9611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,153.9470,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,567.7859,27.6866,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,2753.7102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,162.2703,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,509.0,567.7856,27.6456,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,-4.9222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Drop gazes with GazeX or GazeY equal to NaN. This because these gazes are not mapped to any AOI
data = data.dropna(subset=['GazeX','GazeY'])


# Keep only relevant colomns
saccadeDataWithAOIs = data[['SacID', 'Saccade Start',
       'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']+AOIS].copy(deep=True)



# Map Saccades to AOIs
"""
Goal: 
- Map saccades to AOIs
Context: 
- A saccade comprises several gazes.
- Following the NoteBook "1. Gaze Projections and AOIs Mapping", each gaze point is mapped to an AOI.
Problem: 
- Gazes belonging to the same saccade might not necessarily be mapped to the same AOI.
Solution: 
- When mapping saccades to AOIs, choose the AOI which is mapped to the majority of the gazes composing the saccade
"""                                
saccadeDataWithAOIs = saccadeDataWithAOIs.groupby(['SacID', 'Saccade Start',
       'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction'],dropna=False).agg({aoi: (lambda x: pd.Series.mode(x)[0]) for aoi in AOIS }) 


# Flatten saccadeDataWithAOIs (i.e., a hierarchically indexed pandas.DataFrame)
saccadeDataWithAOIs.columns= [x for x in list(saccadeDataWithAOIs.columns.get_level_values(0))]
saccadeDataWithAOIs = saccadeDataWithAOIs.reset_index()

In [33]:
# Preview saccade data
saccadeDataWithAOIs.head()

,SacID,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
0,1.0,358.0820,399.7678,41.6858,3.4218,134.2210,9141.6289,-6482.1752,211.0207,0,0,0,1,0,0,1,0,0,0,0,0
1,2.0,424.7609,441.4162,16.6553,0.4656,47.6982,NaN,-10.4471,267.5634,0,0,0,0,0,0,1,0,0,0,0,0
2,3.0,474.7339,491.4003,16.6664,0.6259,57.2309,3953.5065,-1038.2112,84.5597,0,0,0,0,0,0,1,0,0,0,0,0
3,4.0,516.3954,566.4076,50.0122,6.5752,200.9994,8448.3586,-9882.1750,224.6962,0,0,0,0,0,0,1,0,0,0,0,0
4,5.0,708.1011,724.7248,16.6237,0.3939,93.2294,7353.6423,NaN,192.9946,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
# export saccade data with AOIs to csv
saccadeDataWithAOIs.to_csv("data/saccadeDataWithAOIsfixationDataWithAOIs.csv",  index=False)